# HCM Crawler
v3
<br>
- Extract DHI features (FEATURES)
- Extra well data info (LABELS)
- ML on features vs labels
- No production correction
- Test data: DanauBubut
- Legacy large-scale maps
- <b> Use v3 of Fan's code</b>

## Import libraries

In [2]:
import time
import pandas as pd
from matplotlib import pyplot as plt
from scipy import ndimage
import numpy as np
import seaborn as sns
from os import listdir
from os.path import isfile, join
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d
import re
import os
import math
import scipy.stats as stats
import pylab as pl
import matplotlib.mlab as mlab

In [3]:
from HCM_QI_analysis_v3 import *

## Set data locations

In [4]:
input_dir_maps_amp_full='input/DaBu/maps/amp/legacy/full/'
input_dir_maps_amp_near='input/DaBu/maps/amp/legacy/near/'
input_dir_maps_amp_far='input/DaBu/maps/amp/legacy/far/'
input_dir_maps_dep='input/DaBu/maps/depth/legacy/'
input_dir_polys='input/DaBu/polygons/legacy/'
input_dir_wells='input/'
input_dir_props='input/'

## Get maps

#### Define all sands and blocks

In [5]:
sands=['M10','M50','AB','A1','A2']

#### Get list of maps that exist

In [6]:
mapfiles_amp_full = [m for m in listdir(input_dir_maps_amp_full) if isfile(join(input_dir_maps_amp_full, m))]
mapfiles_amp_near = [m for m in listdir(input_dir_maps_amp_near) if isfile(join(input_dir_maps_amp_near, m))]
mapfiles_amp_far = [m for m in listdir(input_dir_maps_amp_far) if isfile(join(input_dir_maps_amp_far, m))]
print('Amplitude maps found:')
print('Full :'+str(len(mapfiles_amp_full)))
print('Near :'+str(len(mapfiles_amp_near)))
print('Far  :'+str(len(mapfiles_amp_far)))

Amplitude maps found:
Full :5
Near :5
Far  :5


#### Get list of sand-block combinations that exist

In [7]:
sand_block_combis_sands=[]
sand_block_combis_blocks=[]
sand_block_combis_maps=[]
#
map_type='AMRMS'
#
for thismap in mapfiles_amp_full:
    #
    # if the map is of the type we're interested in:
    if map_type in thismap.split('_')[1:]:
        #
        sand_block_combis_maps.append(thismap)
        #
        found_sand=False
        for sand in sands:
            if sand in thismap.split('_')[1:]:
                sand_block_combis_sands.append(sand)
                found_sand=True
        #
        # flag if any errors
        if not(found_sand): 
            raise Exception("### Sand/Block naming problem for "+str(thismap))
#
# construct df
dm = pd.DataFrame([sand_block_combis_sands,sand_block_combis_maps]).T
dm.columns=['sand','map_amp']
#
# show sand-block combis
dm

,sand,map_amp
0,M10,nldvao_004_92_12PrDMf_psAI_M10_SEED_t_filled_M...
1,M50,nldvao_008_92_12PrDMf_psAI_M50_SEED_t_filled_M...
2,AB,nldvao_013_92_12PrDMf_psAI_AB_SEED_t_filled_MC...
3,A1,nldvao_014_92_12PrDMf_psAI_A1_SEED_t_filled_MC...
4,A2,nldvao_015_92_12PrDMf_psAI_A2_SEED_t_filled_MC...


#### Load the depth maps

In [8]:
mapfiles_depth = [m for m in listdir(input_dir_maps_dep) if isfile(join(input_dir_maps_dep, m))]
print('Depth maps found:')
print(str(len(mapfiles_depth)))

Depth maps found:
5


#### Add the depth maps

In [9]:
mapfiles_depth

['nldvao_004_92_12PrDMf_psAI_M10_SEED_t_filled_MC_Zor_d_2016PSDM_scl100',
 'nldvao_008_92_12PrDMf_psAI_M50_SEED_t_filled_MC_Zor_d_2016PSDM_scl100',
 'nldvao_013_92_12PrDMf_psAI_AB_SEED_t_filled_MC_Zor_d_2016PSDM_scl100',
 'nldvao_014_92_12PrDMf_psAI_A1_SEED_t_filled_MC_Zor_d_2016PSDM_scl100',
 'nldvao_015_92_12PrDMf_psAI_A2_SEED_t_filled_MC_Zor_d_2016PSDM_scl100']

In [10]:
dm['map_dep']=''
for row in range(len(dm)):
    #
    for depthmap in mapfiles_depth:
        if dm.iloc[row]['sand'] in depthmap:
            thisdepthmap=depthmap
    #
    dm.iloc[row,dm.columns.get_loc('map_dep')]=thisdepthmap

In [11]:
dm

,sand,map_amp,map_dep
0,M10,nldvao_004_92_12PrDMf_psAI_M10_SEED_t_filled_M...,nldvao_004_92_12PrDMf_psAI_M10_SEED_t_filled_M...
1,M50,nldvao_008_92_12PrDMf_psAI_M50_SEED_t_filled_M...,nldvao_008_92_12PrDMf_psAI_M50_SEED_t_filled_M...
2,AB,nldvao_013_92_12PrDMf_psAI_AB_SEED_t_filled_MC...,nldvao_013_92_12PrDMf_psAI_AB_SEED_t_filled_MC...
3,A1,nldvao_014_92_12PrDMf_psAI_A1_SEED_t_filled_MC...,nldvao_014_92_12PrDMf_psAI_A1_SEED_t_filled_MC...
4,A2,nldvao_015_92_12PrDMf_psAI_A2_SEED_t_filled_MC...,nldvao_015_92_12PrDMf_psAI_A2_SEED_t_filled_MC...


## Get polygons

## Execute DHI analysis per sand and block

In [14]:
#
isFirst=True
#
for row in range(len(dm)):
    #
    print('\n')
    print('-------------------------------------------------------------------------------')
    print('Analyzing sand '+str(dm.iloc[row]['sand']))
    print('-------------------------------------------------------------------------------')
    #
    map_depth=input_dir_maps_dep+dm.iloc[row]['map_dep']
    map_amp_full=input_dir_maps_amp_full+dm.iloc[row]['map_amp']
    map_amp_near=input_dir_maps_amp_near+dm.iloc[row]['map_amp']
    map_amp_far=input_dir_maps_amp_far+dm.iloc[row]['map_amp']
    polygon=input_dir_polys+dm.iloc[row]['sand']+'.txt'
    properties=input_dir_props+'wells_sands_properties.csv'
    #
    d_tmp=DHI_analysis_v3(map_depth,map_amp_full,map_amp_far,map_amp_near,polygon,properties,dm.iloc[row]['sand'],'tmpQI')
    #
    d_tmp['sand']=dm.iloc[row]['sand']
    #
    if isFirst:
        d_dhi=d_tmp.copy()
        isFirst=False
    else:
        d_dhi=pd.concat([d_dhi,d_tmp]).copy()



-------------------------------------------------------------------------------
Analyzing sand M10
-------------------------------------------------------------------------------


FileNotFoundError: File b'C:/Apps/Ddrive_BSP_Projects/NDI_cmap/Jason_AI_Map_254.cmap' does not exist

In [ ]:
d_dhi